### Small notebook for creating a dataset containing which boroughs are adjacent to one another
### This is then used to create relationships in the KG

In [6]:
import geopandas as gpd
import pandas as pd

# Path to your shapefile (do not include the file extension, just the base name)
shapefile_path = r'../data/raw/gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Ensure the borough name column is correct (often 'NAME' or similar)
borough_col = 'NAME'  # Change if your column is named differently

# Compute adjacency
rows = []
for idx, borough in gdf.iterrows():
    neighbours = gdf[gdf.geometry.touches(borough.geometry)]
    for _, neighbour in neighbours.iterrows():
        rows.append({
            "borough1": borough[borough_col],
            "borough2": neighbour[borough_col]
        })


df = pd.DataFrame(rows)
df

,borough1,borough2
0,Kingston upon Thames,Sutton
1,Kingston upon Thames,Richmond upon Thames
2,Kingston upon Thames,Merton
3,Kingston upon Thames,Wandsworth
4,Croydon,Bromley
...,...,...
131,City of London,Westminster
132,City of London,Camden
133,City of London,Tower Hamlets
134,City of London,Islington


In [7]:
# Remove duplicates (since adjacency is symmetric)
df = df[df['borough1'] < df['borough2']]  # Only keep one direction

# Save to CSV
df.to_csv("../data/processed/neighbouring_boroughs.csv", index=False)